In [4]:
import pandas as pd

In [5]:
import os

In [6]:
os.listdir()

['.ipynb_checkpoints',
 'coursera project',
 'Export_loop-sentiment-pos-neg-train_05112020000000.csv',
 'pp2.JPG',
 'Preprocessing.JPG',
 'Untitled.ipynb',
 'Untitled1.ipynb']

In [7]:
df = pd.read_csv("Export_loop-sentiment-pos-neg-train_05112020000000.csv")

In [8]:
df.head()

,label,text
0,Negative,No one cares about marketing slides - a techni...
1,Positive,Are all three hosts providing storage/capacity...
2,Negative,would loved to had managed to get down to the ...
3,Negative,Vending machine at work is out of Dasani water...
4,Positive,"RT @VMwareEdu: Paul Maritz, CEO and President ..."


In [9]:
'''from textblob import TextBlob'''

'from textblob import TextBlob'

In [10]:

df['text'][0]

'No one cares about marketing slides - a technical HOW TO is a lot better'

In [11]:
import numpy as np

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
count = CountVectorizer()

In [14]:
docs = np.array(['The sun is shining',
                 'The weather is sweet',
                 'The sun is shining, the weather is sweet and one and one is two'
                ])

In [15]:
bag = count.fit_transform(docs)

In [16]:
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [17]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


In [18]:
from sklearn.feature_extraction.text import TfidfTransformer

In [19]:
np.set_printoptions(precision = 2)

In [20]:
tfidf = TfidfTransformer(use_idf = True , norm = 'l2',smooth_idf=True)

In [21]:
print(tfidf.fit_transform(bag).toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


In [22]:
df.loc[0,'text'][-50:]

'keting slides - a technical HOW TO is a lot better'

In [23]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>','',text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',text)
    text = re.sub('[\W]+',' ',text.lower()) +\
        ' '.join(emoticons).replace('-','')
    return text

In [24]:
preprocessor("<a> This is :) a :( test :D")

' this is a test d:) :( :D'

In [26]:
df['text'] = df['text'].apply(preprocessor)

In [30]:
from nltk.stem.porter import PorterStemmer

In [31]:
porter = PorterStemmer()

In [32]:
def tokenizer(text):
    return text.split()

In [42]:
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [43]:
tokenizer('runners like running thus they run')

['runners', 'like', 'running', 'thus', 'they', 'run']

In [44]:
tokenizer_porter('runners like running thus they run')

['runner', 'like', 'run', 'thu', 'they', 'run']

In [45]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pbabbar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [48]:
from nltk.corpus import stopwords

stop = stopwords.words('english')
[w for w in tokenizer_porter('a running likes running and runs a lot')[-10:] if w not in stop]


['run', 'like', 'run', 'run', 'lot']

In [49]:
[w for w in tokenizer_porter('runners like running thus they run')[-10:] if w not in stop]

['runner', 'like', 'run', 'thu', 'run']

In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(strip_accents = None ,
                       lowercase=False,
                       preprocessor=None,
                       tokenizer=tokenizer_porter,
                       use_idf=True,
                       norm='l2',
                       smooth_idf=True)

In [58]:
y = df.label.values
X = tfidf.fit_transform(df.text)

In [59]:
y

array(['Negative', 'Positive', 'Negative', ..., 'Negative', 'Positive',
       'Negative'], dtype=object)

In [60]:
X

<1900x5329 sparse matrix of type '<class 'numpy.float64'>'
	with 44720 stored elements in Compressed Sparse Row format>

In [102]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train , y_test = train_test_split(X,y,random_state =1,test_size=0.5,shuffle=False)

In [103]:
import pickle
from sklearn.linear_model import LogisticRegressionCV

clf = LogisticRegressionCV(cv=5,
                          scoring='accuracy',
                          random_state=0,
                          n_jobs=-1,
                          verbose=3,
                          max_iter=600).fit(X_train,y_train)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   10.4s remaining:   15.7s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   14.1s finished


In [104]:
saved_model = open('saved_model.sav','wb')
pickle.dump(clf, saved_model)
saved_model.close()

In [100]:
filename = 'saved_model.sav'
saved_clf = pickle.load(open(filename,'rb'))

In [101]:
saved_clf.score(X_test,y_test)

0.743421052631579